In [2]:
import cv2
from skimage.filters import threshold_otsu
import numpy as np
from skimage.morphology import (erosion, dilation, opening, closing,  # noqa
                               white_tophat)
from skimage.measure import find_contours
import matplotlib.pyplot as plt
from skimage.draw import rectangle
from math import sqrt
%matplotlib inline 

In [29]:
im_in = cv2.imread("connect.png", cv2.IMREAD_GRAYSCALE)
th, im_th = cv2.threshold(im_in, 220, 255, cv2.THRESH_BINARY_INV)
###################Remove words###############
structureElm =  np.array((
	[1,1],
	[1,1]), dtype="int")
#closed = closing(im_th, structureElm)
#for i in range(5):
#im_th = closing(im_th, structureElm)
cv2.imwrite("closed.png", im_th)

#im2, contours_closed, hierarchy = cv2.findContours(closed, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

True

In [30]:
# Mask used to flood filling.
# Notice the size needs to be 2 pixels than the image.
hImg, wImg = im_th.shape[:2]
area = hImg*wImg
mask = np.zeros((hImg+2, wImg+2), np.uint8)

im_floodfill = im_th.copy()
##################Flood fill corners#####
cv2.floodFill(im_floodfill, mask, (0,0), 255)
cv2.floodFill(im_floodfill, mask, (wImg-1,0), 255)
cv2.floodFill(im_floodfill, mask, (0,hImg-1), 255)
cv2.floodFill(im_floodfill, mask, (wImg-1,hImg-1), 255)

print("corners Filled")

corners Filled


In [35]:
######################cv contours#########
# Invert floodfilled image
im_floodfill_inv = cv2.bitwise_not(im_floodfill)
im2, contours_cv, hierarchy = cv2.findContours(im_floodfill_inv, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
hierarchy = hierarchy[0]
#[Next, Previous, First_Child, Parent]
im_empty = np.ones((hImg, wImg,3), np.uint8) * 255
i=0
for cnt in contours_cv:
    #contours that has at least two children
    child1Idx = hierarchy[i][2]
    hasChildren = child1Idx != -1 and  hierarchy[child1Idx][0] != -1
    if(hasChildren):
        child2Idx = hierarchy[child1Idx][0]
    x,y,w,h = cv2.boundingRect(cnt)
    acceptedArea =    h<=200 and w <=400 and h>10 and w>30
    #this two children are close (letters)
    #check area
    if(hasChildren and acceptedArea):
        #print((w*h)/area)
        x1,y1,w1,h1 = cv2.boundingRect(contours_cv[child1Idx])
        x2,y2,w2,h2 = cv2.boundingRect(contours_cv[child2Idx])
        if(sqrt((x1-x2)**2+((y1-y2)**2)) <= 200):
            cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)
    if(child1Idx != -1 and  hierarchy[child1Idx][0] == -1):
        x1,y1,w1,h1 = cv2.boundingRect(contours_cv[child1Idx])
        if(w1/w >= 0.3):
            cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)
    if(not hasChildren and acceptedArea and w/h >=1.2 ):#and w/h<=3.5 and acceptedArea):
        #print(w/h,w,h)
        cv2.drawContours(im_empty, contours_cv, i, (0,0,0), 1)

    i+=1

#cv2.drawContours(im_empty, contours_cv, -1, (0,0,0), 1)
#print(hierarchy)
cv2.imwrite("contoured.png", im_empty)
#########################################
# Display images.
cv2.imwrite("Thresholded_Image.png", im_th)
cv2.imwrite("Floodfilled_Image.png", im_floodfill)
cv2.imwrite("Inverted_Floodfilled_Image.png", im_floodfill_inv)

True